# Simulación de Partículas con Herencia en p5.js

Este programa demuestra el uso de **herencia clásica en JavaScript** aplicada a una simulación de partículas visuales. 

## Conceptos clave que se utilizarán:

- **Objetos y prototipos**: JavaScript usa prototipos en lugar de clases tradicionales
- **Herencia**: `Ball` heredará propiedades y métodos de `Particle`
- **Getters y setters**: Permiten acceder a propiedades calculadas como `pos2D` y `velo2D`
- **Gradientes radiales**: Efecto visual usando la API nativa de Canvas a través de `drawingContext`

## Estructura del programa:
1. Declaración de variables globales
2. Definición del constructor `Particle` (clase padre)
3. Definición del constructor `Ball` con herencia (clase hija)
4. Inicialización en `setup()`
5. Bucle de dibujo en `draw()`

La siguiente celda declara las variables globales que almacenarán el arreglo de bolas y la cantidad total.

In [1]:
var balls = [];
var numBalls = 10;

## Constructor Particle (Clase Padre)

Aquí definimos el objeto base `Particle` que representa una partícula física abstracta con propiedades fundamentales:

### Propiedades físicas:
- `mass`: Masa de la partícula (siempre positiva, por defecto 1)
- `charge`: Carga eléctrica (positiva, negativa o cero, por defecto 0)
- `x`, `y`: Posición en coordenadas cartesianas (inicialmente 0,0)
- `vx`, `vy`: Componentes de velocidad (inicialmente 0,0)

### Métodos de acceso (getters/setters):
- `pos2D`: Devuelve un vector p5.Vector con la posición actual. Al asignar, descompone el vector en `x` e `y`.
- `velo2D`: Devuelve un vector p5.Vector con la velocidad actual. Al asignar, descompone el vector en `vx` y `vy`.

Esta técnica permite trabajar con vectores de forma conveniente mientras mantiene sincronizados los componentes individuales. No hay duplicación de datos: los vectores se calculan "al vuelo" cada vez que se accede a ellos.

La siguiente celda implementa este constructor y su prototipo con los getters y setters.

In [2]:
function Particle(mass, charge) {
    if (typeof(mass) === 'undefined') mass = 1;
    if (typeof(charge) === 'undefined') charge = 0;
    
    this.mass = mass;
    this.charge = charge;
    this.x = 0;
    this.y = 0;
    this.vx = 0;
    this.vy = 0;
}

Particle.prototype = {
    get pos2D() {
        return createVector(this.x, this.y);
    },
    set pos2D(pos) {
        this.x = pos.x;
        this.y = pos.y;
    },
    get velo2D() {
        return createVector(this.vx, this.vy);
    },
    set velo2D(velo) {
        this.vx = velo.x;
        this.vy = velo.y;
    }
};

## Constructor Ball con Herencia (Clase Hija)

Aquí aplicamos el patrón de **herencia clásica en JavaScript** usando `Object.create()`. Ball extiende Particle añadiendo propiedades visuales.

### Pasos de la herencia:

1. **Llamada al constructor padre**: `Particle.call(this, mass, charge)`  
   Ejecuta el constructor de Particle en el contexto de Ball, heredando sus propiedades.

2. **Propiedades adicionales de Ball**:
   - `radius`: Radio de la bola (por defecto 20 píxeles)
   - `color`: Color en formato hexadecimal (por defecto azul '#0000ff')
   - `gradient`: Booleano que activa/desactiva el efecto de gradiente radial

3. **Establecimiento del prototipo**:
   - `Ball.prototype = Object.create(Particle.prototype)`: Crea un nuevo objeto cuyo prototipo es `Particle.prototype`, permitiendo que Ball acceda a todos los métodos de Particle.
   - `Ball.prototype.constructor = Ball`: Restaura la propiedad constructor para que apunte correctamente a Ball.

4. **Método draw()**:
   - Si `gradient` es true: Usa `drawingContext` para crear un gradiente radial (blanco en el centro, color sólido en el borde).
   - Si `gradient` es false: Usa funciones nativas de p5.js (`fill`, `ellipse`) para dibujar un círculo simple.

La siguiente celda implementa todo este mecanismo de herencia y el método de dibujo.

In [3]:
function Ball(radius, color, mass, charge, gradient) {
    Particle.call(this, mass, charge);
    
    if (typeof(radius) === 'undefined') radius = 20;
    if (typeof(color) === 'undefined') color = '#0000ff';
    if (typeof(gradient) === 'undefined') gradient = false;
    
    this.radius = radius;
    this.color = color;
    this.gradient = gradient;
}

Ball.prototype = Object.create(Particle.prototype);
Ball.prototype.constructor = Ball;

Ball.prototype.draw = function() {
    push();
    
    if (this.gradient) {
        var ctx = drawingContext;
        var grad = ctx.createRadialGradient(
            this.x, this.y, 0,
            this.x, this.y, this.radius
        );
        grad.addColorStop(0, '#ffffff');
        grad.addColorStop(1, this.color);
        
        ctx.fillStyle = grad;
        ctx.beginPath();
        ctx.arc(this.x, this.y, this.radius, 0, 2 * Math.PI);
        ctx.fill();
    } else {
        fill(this.color);
        noStroke();
        ellipse(this.x, this.y, this.radius * 2);
    }
    
    pop();
};

## Función setup(): Inicialización

La función `setup()` se ejecuta **una sola vez** al inicio del programa. Aquí configuramos:

1. **Canvas**: Creamos un lienzo de 800x600 píxeles con `createCanvas()`.

2. **Paleta de colores**: Definimos un arreglo de 10 colores en formato hexadecimal para variedad visual.

3. **Creación de bolas**: Mediante un bucle `for` creamos `numBalls` instancias de Ball con:
   - Radio aleatorio entre 15 y 50 píxeles (`random(15, 50)`)
   - Color seleccionado cíclicamente de la paleta
   - Masa aleatoria entre 0.5 y 3
   - Todas con gradiente activado (`true`)

4. **Posicionamiento aleatorio**: Cada bola se coloca en una posición aleatoria dentro del canvas, asegurando que no se salga de los bordes usando `random(radius, width - radius)`.

Las bolas se almacenan en el arreglo global `balls` para ser dibujadas posteriormente.

La siguiente celda implementa esta inicialización.

In [4]:
function setup() {
    createCanvas(800, 600);
    
    var colors = ['#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff', 
                  '#00ffff', '#ff8800', '#8800ff', '#ff0088', '#88ff00'];
    
    for (var i = 0; i < numBalls; i++) {
        var radius = random(15, 50);
        var color = colors[i % colors.length];
        var mass = random(0.5, 3);
        
        var ball = new Ball(radius, color, mass, 0, true);
        ball.x = random(radius, width - radius);
        ball.y = random(radius, height - radius);
        
        balls.push(ball);
    }
}

## Función draw(): Bucle de Animación

La función `draw()` se ejecuta **aproximadamente 60 veces por segundo**, creando el efecto de animación continua.

### Pasos en cada frame:

1. **Limpiar pantalla**: `background(30)` establece un fondo gris oscuro (valor 30 en escala de grises).

2. **Dibujar todas las bolas**: Recorremos el arreglo `balls` con un bucle `for` y llamamos al método `draw()` de cada instancia. Gracias a la herencia, cada bola tiene acceso a:
   - Sus propiedades propias (`radius`, `color`, `gradient`)
   - Las propiedades heredadas de Particle (`x`, `y`, `mass`, etc.)
   - Los getters/setters del prototipo de Particle (`pos2D`, `velo2D`)

3. **Mostrar información**: Dibujamos texto indicando que se está usando herencia con `Object.create()`.

Este bucle es el corazón de cualquier programa en p5.js: prepara el lienzo, dibuja el estado actual, y se repite indefinidamente.

La siguiente celda implementa este bucle de dibujo.

In [5]:
function draw() {
    background(30);
    
    for (var i = 0; i < balls.length; i++) {
        balls[i].draw();
    }
    
    fill(255);
    textAlign(CENTER);
    textSize(16);
    text("Herencia: Ball extends Particle (Object.create)", width/2, 30);
}

## Visualización

La siguiente celda es una **magia de JupyterLite** (`%show`) que indica al entorno que debe renderizar el canvas de p5.js en la salida de la celda.

Sin esta instrucción, el canvas se crearía pero no sería visible en el notebook.

Ejecuta esta celda para ver el resultado final: múltiples bolas con gradientes radiales de diferentes colores y tamaños, demostrando la herencia entre Ball y Particle.

In [6]:
%show

# Continúa Capitulo 4 pg 82 🔜